In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.pipeline import make_pipeline

from sklearn.preprocessing import StandardScaler

# Load the first database Amazon

In [2]:
df = pd.read_csv("Datasets/amazon/amazon_review_ID.shuf.lrn.csv")
#print(df["Class"].unique())

# Change the attributes from the Class coloum into numbers 

In [3]:
Class_description = []
Class_description = pd.unique(df.Class)

for i in range(len(Class_description)):
    df.Class[df.Class == Class_description[i]] = i+1

In [4]:
scaler = StandardScaler()
scaler.fit(df.drop('Class', axis=1))
scaled_features = scaler.transform(df.drop('Class', axis=1))

In [5]:
#df_test = pd.read_csv('Datasets/amazon/amazon_review_ID.shuf.tes.csv')
X = df.drop('Class', axis=1)
#X = df_test
Y = df.Class.astype(float)

# Sets the parameter alpha for the CLassifierer

In [6]:
h = .02  # step size in the mesh

alphas = np.logspace(-5, 2, 7)
names = ['alpha ' + str(i) for i in alphas]
alphas
modes = ["identity","logistic","tanh","relu"]

In [7]:
X = StandardScaler().fit_transform(X)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.4)

In [ ]:
alphas = [1e-5,1e-4,1e-3,1e-2,1e-1,0,1,10,100]
modes = ["logistic"]
solv = ["adam"]

In [ ]:
#alphas = [1e-5]
#modes = ["identity","logistic","tanh","relu"]
#modes = ["logistic"]
#solv = ["adam","sgd","lbfgs"]
#solv = ["adam"]
erg = []
for i in alphas:
    for j in modes:
        for k in solv:
            clf = MLPClassifier(max_iter=500,activation=j)
            #clf = MLPClassifier(hidden_layer_sizes=(500,500,500), max_iter=300, alpha=i,solver='sgd', verbose=10,  random_state=21,tol=0.000000001,activation=j)
            clf.fit(X_train, Y_train)
            Y_pred = clf.predict(X_test)
            print("alpha: ",i,"mode: ",j,"solver: ",k,"score: ",accuracy_score(Y_test, Y_pred))
            erg.append("alpha:")
            erg.append(i)
            erg.append("mode:")
            erg.append(j)
            erg.append("solver:")
            erg.append(k)
            erg.append("score:")
            erg.append(accuracy_score(Y_test, Y_pred))
    print("")


In [ ]:
al

In [ ]:
clf = MLPClassifier(hidden_layer_sizes=(1000,1000,1000), max_iter=500, alpha=0.0001,solver='sgd', verbose=10,  random_state=21,tol=0.000000001)

In [ ]:
# preprocess dataset, split into training and test part
X = StandardScaler().fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=.4)

x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5
xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                        np.arange(y_min, y_max, h))

In [ ]:
ax = plt.subplot(1,1, 1)

In [ ]:
# just plot the dataset first
cm = plt.cm.RdBu
cm_bright = ListedColormap(['#FF0000', '#0000FF'])
# Plot the training points
ax.scatter(X_train[:, 0], X_train[:, 1], c=y_train, cmap=cm_bright)
# and testing points
ax.scatter(X_test[:, 0], X_test[:, 1], c=y_test, cmap=cm_bright, alpha=0.6)
ax.set_xlim(xx.min(), xx.max())
ax.set_ylim(yy.min(), yy.max())
ax.set_xticks(())
ax.set_yticks(())


In [ ]:

# iterate over classifiers
#for name, clf in zip(names, classifiers):
ax = plt.subplot(1,1, 1)
clf.fit(X_train, y_train)
score = clf.score(X_test, y_test)

# Plot the decision boundary. For that, we will assign a color to each
# point in the mesh [x_min, x_max]x[y_min, y_max].
#if hasattr(clf, "decision_function"):
#    Z = clf.decision_function(np.c_[xx.ravel(), yy.ravel()])
#else:
#    Z = clf.predict_proba(np.c_[xx.ravel(), yy.ravel()])[:, 1]

# Put the result into a color plot
#Z = Z.reshape(xx.shape)
#ax.contourf(xx, yy, Z, cmap=cm, alpha=.8)

# Plot also the training points
ax.scatter(X_train[:, 0], X_train[:, 1], c=y_train, cmap=cm_bright,
            edgecolors='black', s=25)
# and testing points
ax.scatter(X_test[:, 0], X_test[:, 1], c=y_test, cmap=cm_bright,
            alpha=0.6, edgecolors='black', s=25)

ax.set_xlim(xx.min(), xx.max())
ax.set_ylim(yy.min(), yy.max())
ax.set_xticks(())
ax.set_yticks(())
ax.set_title(name)
ax.text(xx.max() - .3, yy.min() + .3, ('%.2f' % score).lstrip('0'),
        size=15, horizontalalignment='right')
plt.show()

In [ ]:
#clf = MLPClassifier(hidden_layer_sizes=(1000,1000,1000), max_iter=500, alpha=0.0001,solver='sgd', verbose=10,  random_state=21,tol=0.000000001)
clf = MLPClassifier(max_iter=100,activation='tanh')
clf

In [ ]:
clf.fit(X_train, Y_train)
Y_pred = clf.predict(X_test)
accuracy_score(Y_test, Y_pred)

In [ ]:
accuracy_score(Y_test, Y_pred)

In [ ]:
cm = confusion_matrix(Y_test, Y_pred)
cm

In [ ]:
sns.heatmap(cm, center=True)
plt.show()

In [ ]:
    classifiers.append(make_pipeline(
                       StandardScaler(),
                       MLPClassifier(solver='logistic', alpha=i,
                                     random_state=1, max_iter=2000,
                                     early_stopping=True,
                                     hidden_layer_sizes=[100, 100])
                       ))